In [5]:
import pandas as pd
import numpy as np

/Applications/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [6]:
housing=pd.read_csv('housing.csv')

In [ ]:
# 1. quicklook
# df.shape:形状
# df.columns:有哪些feature

# df.head():
# df.info():attribute type, nonnull/null values
# df.describe():min,max,std,percentile

# 1-1 numerical
# plot histogram (whole/one at a time):有缺失值无所谓，不影响做图
## 发现问题：（capped/scale/tail-heavy)



# 1-2 categorical
# plot bar
# plt.bar(housing['ocean_proximity'].unique(),height=housing['ocean_proximity'].value_counts())
   
    # categorical variables做计数相关
       # .unique()
       # .nunique()
       # .value_counts()

In [ ]:
# 2. create train/test/validation sets

# 2-1 ordinary
# from sklearn_model_selection import train_test_split
# train_set, test_set = train_test_split(housing, test_size=0.2,
#      random_state=42)

# 2-2 stratified sampling(有些attribute非常重要)
# housing["income_cat"] = pd.cut(housing["median_income"],
#                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
#                               labels=[1, 2, 3, 4, 5])
# housing['income_cat'].hist()



# 2-3 比较和处理
# 列表比较
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

# 除去新添的列
for set_ in (strat_train_set, strat_tset_set):
    set_.drop('income_cat',axis=1,inplace=True)

In [ ]:
# 3. Discover & visualize data (gain insight)

# 3-1 get copy
# housing = strat_train_set.copy()

# 3-2 visualize geographical data: optional
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
      s=housing["population"]/100, label="population", figsize=(10,7),
      c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
  )
  plt.legend()
    
    
# 3.3 correlations
# corr_matrix = housing.corr()

# 查看某一attribute与其他的attributes的关系
# corr_matrix["median_house_value"].sort_values(ascending=False)

# plot correlation matrix

# 3.4 attributes combination

## 4. Prepare

`housing =strat_train_set.drop('median_house_value',axis=1)`
`housing_labels = strat_train_set['median_house_value'].copy()`

### 4-1 Data Cleaning

#### 4-1-1 Missing data
##### 1. get rid of corresponding districts

- **只针对某个列**<br>
`housing.dropna(subset=[''])`

- **丢弃所有为na的行**<br>
`df.dropna(how='all')`<br>

- **丢弃所有为na的列**<br>
`df.dropna(how='all',axis = 1)`<br>

- **留下一部分观测数据**<br>
`df.dropna(thresh=2)`

##### 2. get rid of whole attribute
`housing.drop('total_bedrooms,axis=1)`


##### 3. Fill missing values
1. 直接填充<br>
`median = housing['total_bedrooms']`
`housing['total_bedrooms'].fillna(median, inplace=True)`

2. 利用SimpleImputer<br>
    **这个方法麻烦，但保证你计算得到的填充值一直保留，因为之后会处理test data(evaluate), 系统启用后填充new data**
- 导入module，处理得到纯numerical的列

>`from sklearn.impute import SimpleImputer`<br>
`imputer = SimpleImputer(strategy='median')`<br>
`housing_num = housing.drop('ocean_proximity',axis=1)`

- training & transform<br>
`imputer.fit(housing_num)`<br>
`X = imputer.transform(housing_num)`<br>
    **最终得到一个ndarray**<br>


- 转化成一个dataframe<br>
`housing_tr = pd.DataFrame(X,columns=housing_num.columns,index=housing_num.index)`

 **其他信息**

|对象| Syntax | Description |
|---------|---------|-------|
|instance|imputer.statistics_ | 查看result |
|arg|strategy = 'mean'|num|
|arg|strategy = 'median'|num|
|arg|strategy = 'most_frequent'|num & str|
|arg|strategy = 'constant', fill_value = |num & str|

In [8]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
housing_num = housing.drop('ocean_proximity',axis=1)
imputer.fit(housing_num)

SimpleImputer(strategy='median')

In [12]:
imputer.statistics_

array([-1.1849e+02,  3.4260e+01,  2.9000e+01,  2.1270e+03,  4.3500e+02,
        1.1660e+03,  4.0900e+02,  3.5348e+00,  1.7970e+05])

In [55]:
housing.median().values #得到np.array

array([-1.1849e+02,  3.4260e+01,  2.9000e+01,  2.1270e+03,  4.3500e+02,
        1.1660e+03,  4.0900e+02,  3.5348e+00,  1.7970e+05])

In [56]:
# 注意区别
housing.median() 

longitude               -118.4900
latitude                  34.2600
housing_median_age        29.0000
total_rooms             2127.0000
total_bedrooms           435.0000
population              1166.0000
households               409.0000
median_income              3.5348
median_house_value    179700.0000
dtype: float64

In [19]:
housing2 = housing.copy()
X = imputer.transform(housing_num)

In [22]:
X

array([[-1.2223e+02,  3.7880e+01,  4.1000e+01, ...,  1.2600e+02,
         8.3252e+00,  4.5260e+05],
       [-1.2222e+02,  3.7860e+01,  2.1000e+01, ...,  1.1380e+03,
         8.3014e+00,  3.5850e+05],
       [-1.2224e+02,  3.7850e+01,  5.2000e+01, ...,  1.7700e+02,
         7.2574e+00,  3.5210e+05],
       ...,
       [-1.2122e+02,  3.9430e+01,  1.7000e+01, ...,  4.3300e+02,
         1.7000e+00,  9.2300e+04],
       [-1.2132e+02,  3.9430e+01,  1.8000e+01, ...,  3.4900e+02,
         1.8672e+00,  8.4700e+04],
       [-1.2124e+02,  3.9370e+01,  1.6000e+01, ...,  5.3000e+02,
         2.3886e+00,  8.9400e+04]])

In [58]:
type(X)

numpy.ndarray

In [59]:
housing_tr = pd.DataFrame(X,columns = housing_num.columns,index = housing_num.index)

In [60]:
housing_tr

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [63]:
type(housing_tr) #得到正确的格式

pandas.core.frame.DataFrame

In [62]:
housing_tr.info() #看到missing value已经被补齐

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


##### 4-1-2 Text and categorical attributes
Convert categorical to numerical
1. OrdinalEncoder Class(nearby encoder)
> from sklearn.preprocessing import OrdinalEncoder
  ordinal_encoder=OrdinalEncoder()
  housing_cat_encoded=ordinal_encoder.fit_transform(housing_cat)
  
| syntax|describe|
|--------|------|
|ordinal_encoder.categories_|查看分类(1D array)
  
2. OneHotEncoder (dummy attributes)
> from sklearn.preproces import OneHotEncoder
cat_encoder=OneHotEncoder()
housing_cat_1hot=cat_encoder.fit_transform(housing_cat)

- 得到一个Scipy sparse matrix (only store nonzero elements locations) <br>
    `housing_cat_1hot`
- transform to a np.array<br>
    `housing_cat_1hot.toarray()`

| syntax|describe|
|--------|------|
|cat_encoder.categories_|查看分类(1D array)

3. 根据意义将categorical转化成numerical
- Series的map
- replace

4. embedding (representation learning)

In [36]:
housing['ocean_proximity']

0        NEAR BAY
1        NEAR BAY
2        NEAR BAY
3        NEAR BAY
4        NEAR BAY
           ...   
20635      INLAND
20636      INLAND
20637      INLAND
20638      INLAND
20639      INLAND
Name: ocean_proximity, Length: 20640, dtype: object

In [37]:
housing_cat = housing[['ocean_proximity']]

,ocean_proximity
0,NEAR BAY
1,NEAR BAY
2,NEAR BAY
3,NEAR BAY
4,NEAR BAY
...,...
20635,INLAND
20636,INLAND
20637,INLAND
20638,INLAND


In [57]:
# 必须使用2-d array，这个例子给的是一个series
housing_cat = housing['ocean_proximity']
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

ValueError: Expected 2D array, got 1D array instead:
array=['NEAR BAY' 'NEAR BAY' 'NEAR BAY' ... 'INLAND' 'INLAND' 'INLAND'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [69]:
housing_cat_num=housing[['ocean_proximity']]

In [70]:
housing_cat_num.value_counts()

ocean_proximity
<1H OCEAN          9136
INLAND             6551
NEAR OCEAN         2658
NEAR BAY           2290
ISLAND                5
dtype: int64

In [72]:
type(housing_cat_num)

pandas.core.frame.DataFrame

##### 4-1-3 Custom transformer

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

#一般都是没有其他父类，用object基类，这个例子继承了两个其他父类（base class），因为是sklearn要有三个method，
#fit(return self), transform(),fit_transform(), 其中TransformerMixin提供fit_transform()method,
#BaseEstimator(avoid *args/**kargs in the constructor),另外获得两个method，
#(get_params()/set_params()for automatic hyperparameter tuning)

class CombinedAttributesAdder(BaseEstimator, TransformerMixin): 
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs, add_bedrooms_per_room is the only hyperparameter
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None): #固定搭配 self.fit(X)
        return self  # nothing else to do
    def transform(self, X): #固定搭配 self.transform(X)
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

#create instance，CombinedAttributesAdder是一个constructor
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
#self.trandsform(X)
housing_extra_attribs = attr_adder.transform(housing.values)

In [79]:
from sklearn.base import BaseEstimator, TransformerMixin

In [80]:
BaseEstimator?

In [85]:
class Transtolower(BaseEstimator,TransformerMixin):
    def __init__(self,tr = True):
        self.tr = tr
    def fit(self,X, y =None):
        return self
    def transform(self,X):
        X = X.str.lower()

In [82]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
   ....:                               'Pastrami', 'corned beef', 'Bacon',
   ....:                               'pastrami', 'honey ham', 'nova lox'],
   ....:                      'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [83]:
trans = Transtolower(tr=)

TypeError: __init__() missing 1 required positional argument: 'r'